<a href="https://colab.research.google.com/github/isaakcueva/Regresion_Lineal_Var_Bucket/blob/main/Regresi%C3%B3n_Lineal_con_Variables_Tipo_Bucket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarea: Regresión Lineal con Variables Tipo Bucket

# Nombre: Isaac Cueva

# Fecha: 13/05/2025

![Descripción de la imagen](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRP9d63jC144Usqa_NjecnoWcJXV60JuFTJLA&s)

###Google Colab Setup

In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [ ]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context.
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Provides findspark.init() to make pyspark importable as a regular library.
import findspark
findspark.init()

In [ ]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Regresión Lineal") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
spark

#TAREAS

1. Creando buckets para departure time

- Convierta el flight departure time de valores numéricos entre 0 a 24, a valores en buckets. Use buckets de 3 horas [0,3,6,...,21,24]. Guarde este datos como la variable depart_bucket.



In [ ]:
# Cargar el dataset
file_path = 'flights-larger.csv'
flights_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Mostrar las primeras filas para entender la estructura
flights_df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102|   NA|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



In [ ]:
# Imprimir el esquema del DataFrame
flights_df.printSchema()


root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col, sum

# Verificar valores nulos por cada columna
null_counts = flights_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in flights_df.columns])

# Mostrar la cantidad de valores nulos por columna
null_counts.show()


+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
|  0|  0|  0|      0|     0|  0|   0|     0|       0|    0|
+---+---+---+-------+------+---+----+------+--------+-----+



In [ ]:
from pyspark.sql.functions import col

# Convertir 'delay' a tipo numérico (float), lo que convierte los valores no numéricos a null
flights_df = flights_df.withColumn("delay", col("delay").cast("float"))

# Verificar el tipo de la columna después de la conversión
flights_df.printSchema()


root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: float (nullable = true)



In [ ]:
# Verificar la cantidad de valores nulos en la columna 'delay'
null_delay_count = flights_df.filter(col("delay").isNull()).count()
print(f"Cantidad de valores nulos en 'delay': {null_delay_count}")


Cantidad de valores nulos en 'delay': 16711


In [ ]:
# Calcular la mediana de la columna 'delay'
median_delay = flights_df.approxQuantile("delay", [0.5], 0.01)[0]
print(f"Mediana de 'delay': {median_delay}")

# Reemplazar los valores nulos en la columna 'delay' por la mediana calculada
flights_df_clean = flights_df.fillna({'delay': median_delay})

# Verificar si aún quedan valores nulos en la columna 'delay'
null_delay_count_clean = flights_df_clean.filter(col("delay").isNull()).count()
print(f"Cantidad de valores nulos en 'delay' después de la limpieza: {null_delay_count_clean}")


Mediana de 'delay': 15.0
Cantidad de valores nulos en 'delay' después de la limpieza: 0


In [ ]:
# Eliminar las filas con valores nulos en 'delay' si es necesario
flights_df_clean = flights_df_clean.filter(col("delay").isNotNull())

# Verificar las primeras filas después de la limpieza
flights_df_clean.show(5)


+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51| 27.0|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| 15.0|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|-19.0|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365| 60.0|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85| 22.0|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import when

# Crear buckets para departure time
flights_df_clean = flights_df_clean.withColumn(
    'depart_bucket',
    when((col('depart') >= 0) & (col('depart') < 3), '0-3')
    .when((col('depart') >= 3) & (col('depart') < 6), '3-6')
    .when((col('depart') >= 6) & (col('depart') < 9), '6-9')
    .when((col('depart') >= 9) & (col('depart') < 12), '9-12')
    .when((col('depart') >= 12) & (col('depart') < 15), '12-15')
    .when((col('depart') >= 15) & (col('depart') < 18), '15-18')
    .when((col('depart') >= 18) & (col('depart') < 21), '18-21')
    .when((col('depart') >= 21) & (col('depart') < 24), '21-24')
    .otherwise('Unknown')  # Valor por defecto si no cae en ningún rango
)

# Verificar el resultado
flights_df_clean.select('depart', 'depart_bucket').show(5)


+------+-------------+
|depart|depart_bucket|
+------+-------------+
|  8.18|          6-9|
|  15.5|        15-18|
|  7.17|          6-9|
| 21.17|        21-24|
| 12.92|        12-15|
+------+-------------+
only showing top 5 rows



- Realice one-hot encoding a los valores en buckets calculados. Guarde estos datos como la variable depart_dummy.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Realizar el StringIndexer para convertir las categorías en índices
indexer_depart = StringIndexer(inputCol="depart_bucket", outputCol="depart_bucket_index")

# Realizar el OneHotEncoder para crear las variables binarias
encoder_depart = OneHotEncoder(inputCol="depart_bucket_index", outputCol="depart_dummy")

# Crear el pipeline con los pasos de indexación y codificación
pipeline_depart = Pipeline(stages=[indexer_depart, encoder_depart])

# Aplicar el pipeline a los datos
model_depart = pipeline_depart.fit(flights_df_clean)
flights_df_clean = model_depart.transform(flights_df_clean)

# Verificar las nuevas columnas generadas
flights_df_clean.select('depart', 'depart_bucket', 'depart_bucket_index', 'depart_dummy').show(5)


+------+-------------+-------------------+-------------+
|depart|depart_bucket|depart_bucket_index| depart_dummy|
+------+-------------+-------------------+-------------+
|  8.18|          6-9|                4.0|(7,[4],[1.0])|
|  15.5|        15-18|                0.0|(7,[0],[1.0])|
|  7.17|          6-9|                4.0|(7,[4],[1.0])|
| 21.17|        21-24|                5.0|(7,[5],[1.0])|
| 12.92|        12-15|                1.0|(7,[1],[1.0])|
+------+-------------+-------------------+-------------+
only showing top 5 rows



In [ ]:
# Realizar el StringIndexer para convertir las categorías en índices
indexer_org = StringIndexer(inputCol="org", outputCol="org_index")

# Realizar el OneHotEncoder para crear las variables binarias
encoder_org = OneHotEncoder(inputCol="org_index", outputCol="org_dummy")

# Crear el pipeline con los pasos de indexación y codificación
pipeline_org = Pipeline(stages=[indexer_org, encoder_org])

# Aplicar el pipeline a los datos
model_org = pipeline_org.fit(flights_df_clean)
flights_df_clean = model_org.transform(flights_df_clean)

# Verificar que se haya creado la columna 'org_dummy'
flights_df_clean.select('org', 'org_index', 'org_dummy').show(5)


+---+---------+-------------+
|org|org_index|    org_dummy|
+---+---------+-------------+
|ORD|      0.0|(7,[0],[1.0])|
|ORD|      0.0|(7,[0],[1.0])|
|ORD|      0.0|(7,[0],[1.0])|
|JFK|      2.0|(7,[2],[1.0])|
|SJC|      5.0|(7,[5],[1.0])|
+---+---------+-------------+
only showing top 5 rows



2. Modelo de regresión lineal simple para predecir flight delay

- Las variables predictoras serán. km, org_dummy, depart_dummy.

- Ensamble las variables predictoras en un vector ensamblador llamado features.

- Divida el dataset en datos de entrenamiento y evaluación con un radio de 8 a 2.

- Cree un modelo de regresión multivariable simple que permita predecir la duración de un vuelo con los datos de entrenamiento.

- Determine las predicciones sobre los datos de evaluación.
Calcule el valor del R^2 con los datos de evaluación.

- Compare este modelo con el modelo que realizó anteriormente. ¿Cuál modelo es mejor?

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Crear el VectorAssembler con las variables predictoras
assembler = VectorAssembler(inputCols=['mile', 'org_dummy', 'depart_dummy'],
                            outputCol="features")

# Aplicar el assembler
flights_df_clean = assembler.transform(flights_df_clean)

# Verificar el DataFrame con el vector de características
flights_df_clean.select('features', 'delay').show(5)


+--------------------+-----+
|            features|delay|
+--------------------+-----+
|(15,[0,1,12],[157...| 27.0|
|(15,[0,1,8],[466....| 15.0|
|(15,[0,1,12],[738...|-19.0|
|(15,[0,3,13],[224...| 60.0|
|(15,[0,6,9],[386....| 22.0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# Dividir el dataset en datos de entrenamiento y evaluación (80% entrenamiento, 20% evaluación)
train_df, test_df = flights_df_clean.randomSplit([0.8, 0.2], seed=42)

# Verificar las dimensiones de los conjuntos
train_df.count(), test_df.count()


(219887, 55113)

#Modelo De Regresión Lineal Simple

In [ ]:
from pyspark.ml.regression import LinearRegression

# Crear el modelo de regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="delay")

# Entrenar el modelo
lr_model = lr.fit(train_df)

# Realizar las predicciones
predictions = lr_model.transform(test_df)

# Calcular el valor R^2
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="delay", predictionCol="prediction", metricName="r2")

r2_lr = evaluator.evaluate(predictions)
print(f"R^2 para el modelo de regresión lineal: {r2_lr}")


R^2 para el modelo de regresión lineal: 0.04411356197227101


Compare este modelo con el modelo que realizó anteriormente. ¿Cuál modelo es mejor?

A comparación del modelo de Regresión Lineal elaborado la práctica anterior,el cual fue de 0,017, este presenta un aumento leve aunque igual muy bajo. La razón principal es la falta de variables para poder predecir el retraso de vuelos adecuadamente.

#Modelo De Regresión Múltiple

2. Modelo de regresión polinómica de grado 2 para predecir flight delay

- Cree un modelo de regresión polinómica de grado dos, usando las variables predictoras  km, org_dummy, depart_dummy.

- Determine las predicciones sobre los datos de evaluación.

- Calcule el valor del R^2 con los datos de evaluación.

- Compare este modelo con el modelo que realizó anteriormente. ¿Cuál modelo es mejor?

In [ ]:
# Aplicar transformación polinómica de grado 2 a las características para el conjunto de entrenamiento
poly_expansion = PolynomialExpansion(inputCol="features", outputCol="poly_features", degree=2)

# Transformar los datos de entrenamiento y de prueba
train_df_poly = poly_expansion.transform(train_df)
test_df_poly = poly_expansion.transform(test_df)

# Verificar que se haya creado la columna 'poly_features'
train_df_poly.select('features', 'poly_features').show(5)
test_df_poly.select('features', 'poly_features').show(5)


+--------------------+--------------------+
|            features|       poly_features|
+--------------------+--------------------+
|(15,[0,3,11],[247...|(135,[0,1,9,10,13...|
|(15,[0,10],[3711....|(135,[0,1,65,66,7...|
|(15,[0,2,13],[258...|(135,[0,1,5,6,8,1...|
|(15,[0,2,12],[258...|(135,[0,1,5,6,8,9...|
|(15,[0,3,8],[2586...|(135,[0,1,9,10,13...|
+--------------------+--------------------+
only showing top 5 rows

+--------------------+--------------------+
|            features|       poly_features|
+--------------------+--------------------+
|(15,[0,13],[2486....|(135,[0,1,104,105...|
|(15,[0,3,8],[2475...|(135,[0,1,9,10,13...|
|(15,[0,1,8],[867....|(135,[0,1,2,3,4,4...|
|(15,[0,1,11],[119...|(135,[0,1,2,3,4,7...|
|(15,[0,1,13],[733...|(135,[0,1,2,3,4,1...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# Crear el modelo de regresión lineal para las características transformadas
lr_poly = LinearRegression(featuresCol="poly_features", labelCol="delay")

# Entrenar el modelo con los datos de entrenamiento transformados
lr_poly_model = lr_poly.fit(train_df_poly)

# Realizar las predicciones en el conjunto de prueba transformado
predictions_poly = lr_poly_model.transform(test_df_poly)

# Calcular el valor R^2 para el modelo polinómico
r2_poly = evaluator.evaluate(predictions_poly)
print(f"R^2 para el modelo de regresión polinómica de grado 2: {r2_poly}")


R^2 para el modelo de regresión polinómica de grado 2: 0.051993631619496994


In [ ]:
print(f"R^2 de la regresión lineal simple: {r2_lr}")
print(f"R^2 de la regresión polinómica de grado 2: {r2_poly}")


R^2 de la regresión lineal simple: 0.04411356197227101
R^2 de la regresión polinómica de grado 2: 0.051993631619496994


Los resultados muestran que el modelo de regresión polinómica de grado 2 es levemente mejor pero aún insuficiente para ser válido. Aunque ambos valores son bajos, el modelo polinómico tiene un rendimiento ligeramente mejor debido a que captura relaciones no lineales entre las variables predictoras y la variable objetivo (delay), lo cual permite que el modelo ajuste mejor los datos complejos que no pueden ser capturados por una regresión lineal simple.

#Exportar la Notebook

- El mejor modelo es el de Regresión Múltiple, por lo que vamos a exportarlo

In [ ]:
# Guardar el modelo de regresión polinómica
lr_poly_model.save("/mnt/data/trained_model")


In [ ]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Guardar el modelo de regresión polinómica en Google Drive
model_save_path = '/content/drive/My Drive/trained_model'
lr_poly_model.save(model_save_path)
